In [5]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
from pprint import pprint
from datetime import datetime
import re
import os
from selenium import webdriver


In [2]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


In [3]:
url = "http://www.channelcrawler.com/eng"
browser.visit(url)
browser.is_element_present_by_css("input#queryMinSubs", wait_time=1)
html = browser.html


In [4]:
channel_soup.find('input', id='queryMinSubs')

<input class="form-control" id="queryMinSubs" name="data[query][min_subs]" placeholder="Min" type="text"/>

In [11]:
minSubs = 10000000
driver = webdriver.Chrome()
driver.get(url)
sbox = driver.find_element_by_id('queryMinSubs')
sbox.send_keys(minSubs)
submit = driver.find_element_by_class_name('submitbutton')
submit.click()


In [16]:
html = driver.page_source
channel_soup = bs(html, 'html.parser')
channel_divs = channel_soup.find_all('div', class_="channel")

In [50]:
smalls = channel_divs[1].find_all('small')
smalls[1].get_text()
    

'\n\t\t\t\t\t11,430,063 Subscribers\n\t\t\t\t\t2095 Videos\n\t\t\t\t\tJoin Date: 19.10.2015\t\t\t\t'

In [44]:
list_10M_subs = []
for channel in channel_divs:
    channel_dict = {}
    title_anchor = channel.find('a')
    channel_dict['title'] = title_anchor.get_text()
    channel_dict['channel_url'] = title_anchor['href']
    anchors = channel.find_all('a')
    channel_dict['thumbnail'] = anchors[1].find('img')['src']
    list_10M_subs.append(channel_dict)
    
list_10M_subs

[{'title': 'Maroon 5',
  'channel_url': 'http://www.youtube.com/channel/UCBVjMGOIkavEAhyqpxJ73Dw',
  'thumbnail': 'https://yt3.ggpht.com/a-/AAuE7mA3aBvEoB6djY9q0xt6P-XI9bNI7xx9gHimug=s88-mo-c-c0xffffffff-rj-k-no'},
 {'title': 'Technical Guruji',
  'channel_url': 'http://www.youtube.com/channel/UCOhHO2ICt0ti9KAh-QHvttQ',
  'thumbnail': 'https://yt3.ggpht.com/a-/AAuE7mBrECozW9Jfp_sLjqKFpYtBIEEqZHv9OcLF8A=s88-mo-c-c0xffffffff-rj-k-no'},
 {'title': 'PewDiePie',
  'channel_url': 'http://www.youtube.com/channel/UC-lHJZR3Gqxm24_Vd_AJ5Yw',
  'thumbnail': 'https://yt3.ggpht.com/a-/AAuE7mAPBVgUYqlLw9SvJyKAVWmgkQ2-KrkgSv4_5A=s88-mo-c-c0xffffffff-rj-k-no'},
 {'title': 'Jimmy Kimmel Live',
  'channel_url': 'http://www.youtube.com/channel/UCa6vGFO9ty8v5KZJXQxdhaw',
  'thumbnail': 'https://yt3.ggpht.com/a-/AAuE7mALYvrClnYCRq2J4Qp8qupyOEp0XiWxW1LkyA=s88-mo-c-c0xffffffff-rj-k-no'},
 {'title': 'YRF',
  'channel_url': 'http://www.youtube.com/channel/UCbTLwN10NoCU4WDzLf1JMOA',
  'thumbnail': 'https://yt3.